In [7]:
import json
import requests

In [1]:
def session_token():
    url = "https://www.mca.gov.in/bin/mca/login"
    headers = {
        "Origin": "www.mca.gov.in",
        "Content-Length": "231",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.127 Safari/537.36",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Accept": "*/*",
        "X-Requested-With": "XMLHttpRequest",
        "Sec-Ch-Ua-Platform": "Windows"
    }
    data='data=Ut8pBOc0RSM6iYqffqN1oiyj9WvJGiJEVVlWEbrO16gkiUbYKGssebXlM%2Bcc1l6%2FRa1rzqxyKlKRNXs81w%2F3ugdZoQnhpw6y744KTYxnEt4WKzZc8O2M6nF8ax9u%2FfCNBZICQctC5RRX50JeeMBEZBFLYKy9JK2dm06hApKLQKh2fSRa6WGE%2FLdZXzjX3USndrQ0%2FvDXb%2FnsW4D1oidHaw%3D%3D'
    response = requests.post(url, headers=headers, data=data)
    set_cookie_value = response.headers.get('Set-Cookie')
    start_index = set_cookie_value.find("session-token=") + len("session-token=")
    end_index = set_cookie_value.find(";", start_index)
    session_token = set_cookie_value[start_index:end_index]
    return session_token

In [4]:
def spice_dashboard(session_token,srn):
    url = 'https://www.mca.gov.in/bin/spiceDashboard'
    params = {'srn': f'{srn}'}
    headers = {
        'Cookie': 'cookiesession1=678B2869A3F37E351C741CDEA2BC2CAE; alertPopup=true; session-token={session_token}',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'X-Requested-With': 'XMLHttpRequest',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Te': 'trailers'
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Check if the request was successful
        # print("Response:")
        ref_no=json.loads(response.text)['linkedForms'][1]['referenceNumber']
        # print(response.text)  # Print the response content if needed
        return ref_no
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
    
    

In [5]:
def page_next(session_token,ref_no):
    url = 'https://www.mca.gov.in/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer.af.dermis'
    headers = {
        'Cookie': f'cookiesession1=678B2869A3F37E351C741CDEA2BC2CAE; alertPopup=true; session-token={session_token}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0',
        'Accept': 'text/plain, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        # 'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/e-filing/incorporation/SpicePartB.html',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Csrf-Token': 'undefined',
        'Origin': 'https://www.mca.gov.in',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Te': 'trailers'
    }

    payload = {
        'functionToExecute': 'invokeFDMOperation',
        'formDataModelId': '/content/dam/formsanddocuments-fdm/mca-aem-forms/spice-part-b/spice-b-prefill',
        'input': f'{{"Model0": {{"requestBody": {{"userId": "", "referenceNumber": "{ref_no}", "srn": ""}}}}}}',

        'operationName': 'POST /spicepartb/services/getSRNDetails/1.0.0',
        # 'guideNodePath': '/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer/rootPanel/items/panel_437033673/items/panel_651281723_copy_1876865449/items/mca_button_copy_33766249'
        'guideNodePath': '/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer/rootPanel/items/panel_2001397683/items/panel_651281723/items/mca_button_copy_33766249'

    }

    response = requests.post(url, headers=headers, data=payload)
    intid=json.loads(response.text)['integrationId']
    formIntId=json.loads(response.text)['formIntegrationId']
    return intid,formIntId

In [10]:
def save(session_token,payload):
    url = "https://www.mca.gov.in/bin/mca-gov/SpicePartB_SAVESUBMIT"

    headers = {
        "Cookie": f"cookiesession1=678B28691F6518167EE395808C55BE5E; alertPopup=true; session-token=e{session_token}",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Referer": "https://www.mca.gov.in/content/mca/global/en/mca/e-filing/incorporation/SpicePartB.html",
        "X-Requested-With": "XMLHttpRequest",
        "Content-Type": "multipart/form-data",
        "Content-Length": "3692",
        "Origin": "https://www.mca.gov.in",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Te": "trailers"
    }


    multipart_data = MultipartEncoder(fields={
        'data': (payload),
        'action': 'savesubmit',
        'operation': 'Save',
        'serveAction': 'saveForm',
        'referenceNumber': '',
        'formId': ''
    })

    headers['Content-Type'] = multipart_data.content_type
    response = requests.post(url, headers=headers, data=multipart_data)

    print(response.text)

In [13]:
def pincode(session_token,pincode):
    import requests

    url = 'https://www.mca.gov.in/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer.af.dermis'
    headers = {
        'Cookie': f'cookiesession1=678B286955776DB15177E7C622CE6403; alertPopup=true; session-token={session_token}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0',
        'Accept': 'text/plain, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/e-filing/incorporation/SpicePartB.html',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Csrf-Token': 'undefined',
        'Origin': 'https://www.mca.gov.in',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Te': 'trailers'
    }

    
    payload = {
        'functionToExecute': 'invokeFDMOperation',
        'formDataModelId': '/content/dam/formsanddocuments-fdm/mca-aem-forms/spice-part-b/pincode',
        'input': f'{{"pinCode":"{pincode}"}}',
        'operationName': 'GET /common/service/lookup/pindetails/1.0.0',
        'guideNodePath': '/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer/rootPanel/items/panel_2001397683/items/panel_1902152287/items/guidetextbox_copy_11_762323347'
    }

    response = requests.post(url, headers=headers, data=payload)

    # print(response.text)
    # print(response.status_code)
    a=json.loads(response.text)['data']
    area=[i['Area'] for i in a]
    state=a[0]['State']
    city=a[0]['city']
    district=a[0]['District']
    state=a[0]['State']
    return area,state,city,district,state

In [15]:
def arealocality(session_token,state):
    url = 'https://www.mca.gov.in/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer.af.dermis'
    headers = {
        'Cookie': f'cookiesession1=678B286955776DB15177E7C622CE6403; alertPopup=true; session-token={session_token}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0',
        'Accept': 'text/plain, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/e-filing/incorporation/SpicePartB.html',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Csrf-Token': 'undefined',
        'Origin': 'https://www.mca.gov.in',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Te': 'trailers'
    }

    payload = {
        'functionToExecute': 'invokeFDMOperation',
        'formDataModelId': '/content/dam/formsanddocuments-fdm/mca-aem-forms/spice-part-b/newroclist',
        'input': '{"Type":"MCA_ROC_LIST","Name":'+f"'{state}'"+'}',
        'operationName': 'GET /common/service/lookup/form15/val/1.0.0',
        'guideNodePath': '/content/forms/af/mca-aem-forms/spice-part-b/spice-plus-part-b/jcr:content/guideContainer/rootPanel/items/panel_2001397683/items/panel_1902152287/items/panel_1966601478/items/guidenumericbox_6907'
    }

    response = requests.post(url, headers=headers, data=payload)

    # print(response.text)
    # print(response.status_code)
    reg=json.loads(response.text)['data'][0]['name']
    return reg


In [ ]:
def attachfile(session_token,file_path,label):
    import requests

    url = 'https://www.mca.gov.in/bin/mca/dms/addservice'
    # file_path = '/path/to/your/file.pdf'
    # session_token = 'your_session_token'

    # Set the headers
    headers = {
        'Cookie': f'cookiesession1=678B286955776DB15177E7C622CE6403; alertPopup=true; session-token={session_token}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0',
        'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/e-filing/incorporation/SpicePartB.html',
        'X-Requested-With': 'XMLHttpRequest',
        'Csrf-Token': 'undefined',
        'Origin': 'https://www.mca.gov.in',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Te': 'trailers'
    }

    # Open the file in binary mode
    with open(file_path, 'rb') as file:
        # Create the multipart form-data payload
        payload = {
            'file': ('AA290623000151T_RC13062023.pdf', file, 'application/pdf'),
            'folder': '',
            'filemetadata' : f'{{"form":"SPICE + Part B","srn":"","documentType":"Attachment","module":"SPICE + Part B","userid":"SPICE + Part B","applicationId":"","attachmentCategory":"pdf","attachmentLabel":"{label}"}}'

        }

        # Send the POST request
        response = requests.post(url, headers=headers, files=payload)

    # Check the response
    if response.status_code == 200:
        print('File uploaded successfully')
    else:
        print('Failed to upload the file')


In [8]:
payload1=r'''{"requestBody":{"formData":{"aoaEntrenched":"N","Havingsharecapital":"Having Share Capital","totalAuthorizedShareCapital":"200000","totalClassifiedAuthorizedShareCapital":"100000","totalSubscribedShareCapital":"100000","totalUnclassifiedAuthorizedShareCapital":"100000","equitynumberofClasses":"1","preferencenumberofClasses":"0","regaddresscountry":"India","NonIndividualCountWithoutDIN":"0","SpicePartBINDVSubscribersHavingDin":[{"INDVSubscribersEquityShares":[{"sharecategory":"Equity Share"}],"INDVSubscribersPreferenceShares":[{"sharecategory":"Preference Share"}]}],"SpicePartBINDVSubscribersNotHavingDin":[{"SpicePartBINDVsubscribersPermAddress":{"addressType":"Permanent Address"},"SpicePartBINDVsubscribersPresAddress":{"addressType":"Present Address"},"INDVSubscribersEquityShares":[{"sharecategory":"Equity Share"}],"INDVSubscribersPreferenceShares":[{"sharecategory":"Preference Share"}]}],"SpicePartBSubscribersHavingDin":[{"SubscriberType":"Subscriber CUM Director DIN","category":"Individual First subscribers cum Directors","SpicePartBSubscribersEquityShares":[{"sharecategory":"Equity Share"}],"SpicePartBSubscribersPreferenceShares":[{"sharecategory":"Preference Share"}],"SpicePartBSubsDeclaration":[{"shareholdingPercentage":"0"}]}],"SpicePartBSubscribersNotHavingDin":[{"category":"Individual First subscribers cum Directors","SubscriberType":"Subscriber CUM Director No DIN","SpicePartBSubscribersPermAddress":{"addressType":"Permanent Address"},"SpicePartBSubscribersPresAddress":{"addressType":"Present Address"},"SpicePartBSubscribersEquityShares":[{"sharecategory":"Equity Share"}],"SpicePartBSubscribersPreferenceShares":[{"sharecategory":"Preference Share"}],"SpicePartBSubsDeclaration":[{"shareholdingPercentage":"0"}]}],"SpicePartBDirectorsHavingDin":[{"category":"Directors","SubscriberType":"Directors Having DIN","SpicePartBSubsDeclaration":[{"shareholdingPercentage":"0"}]}],"SpicePartBDirectorsNotHavingDin":[{"category":"Directors","SubscriberType":"Directors Not Having DIN","SpicePartBSubscribersPermAddress":{"addressType":"Permanent Address"},"SpicePartBSubscribersPresAddress":{"addressType":"Present Address"},"SpicePartBSubsDeclaration":[{"shareholdingPercentage":"0"}]}],"formIntegrationId":"1-4C351SZ","SpicePartBShares":[{"classShare":"1","authorizedcapitalsharenumber":"10000","subscribedcapitalsharenumber":"10000","authorizedcapitalnominalamountpershare":"10","subscribedcapitalnominalamountpershare":"10","authorizedcapitaltotalamount":"100000","subscribedcapitaltotalamount":"100000","sharecategory":"Equity Share"}],"spicePartBNonINDVSubscribers":[]},"status":"Draft/Pending Submission","formDescription":"For Spice+PartB","formName":"SPICE + Part B","formVersion":"01","userId":"BIPULKUMARSINGH6690@GMAIL.COM","integrationId":"1-4C351PA","referenceNumber":"1-9437997790","formAttachment":[],"operation":"Save","action":"savesubmit","formIntegrationId":"1-4C351SZ","srn":""}}'''
payload1_json=json.loads(payload1)

In [11]:
payload1_json['requestBody']['formData']['SpicePartBShares'][0]['subscribedcapitalsharenumber']='10000'
payload1_json['requestBody']['formData']['SpicePartBShares'][0]['authorizedcapitalnominalamountpershare']='10'
payload1_json['requestBody']['formData']['SpicePartBShares'][0]['authorizedcapitaltotalamount']
payload1_json['requestBody']['formData']['SpicePartBShares'][0]['authorizedcapitalsharenumber']='10000'
payload1_json['requestBody']['formData']['SpicePartBShares'][0]['authorizedcapitaltotalamount']=''
payload1_json['requestBody']['formData']['SpicePartBShares'][0]['subscribedcapitaltotalamount']=''
payload1_json['requestBody']['formData']['Havingsharecapital']=''
payload1_json['requestBody']['userId']=inp['userId']
payload1_json['requestBody']['referenceNumber']=ref_no
payload1_json['requestBody']['formData']['formIntegrationId']=formIntId
payload1_json['requestBody']['formIntegrationId']=formIntId
payload1_json['requestBody']['integrationId']=intid
payload1_json=json.dumps(payload1_json)

NameError: name 'inp' is not defined